In [1]:
# dependencies
from config import api_key
import pandas as pd
import numpy as np
from citipy import citipy
import matplotlib.pyplot as plt
import matplotlib
import random
import requests

import time

#for grid styling
plt.style.use("seaborn")


In [2]:
# get lat/long zones
lat_zone = np.arange(-90, 90, 10)
long_zone = np.arange(-180, 180, 10)

In [3]:
# new dataframe for coordinates
cities_df = pd.DataFrame()

cities_df["latitude"] = ""
cities_df["longitude"] = ""


In [ ]:

for x in lat_zone:
    for y in long_zone:
        x_values = list(np.arange(x, x+20, 0.01))
        y_values = list(np.arange(y, y+20, 0.01))
        lat = random.sample(x_values, 50)
        long = random.sample(y_values, 50)
        lat_samples = [(x+dec_lat) for dec_lat in lat]
        lon_samples = [y+dec_lon for dec_lon in long]
        cities_df = cities_df.append(pd.DataFrame.from_dict({"latitude":lat, "longitude":long}))
cities_df=cities_df.reset_index(drop=True)
cities_df.head()

,latitude,longitude
0,-84.90,-169.81
1,-80.97,-166.70
2,-73.47,-163.28
3,-74.70,-177.99
4,-87.36,-175.06


In [ ]:
# citipy to find nearest city
cities_df["closest city name"] = ""
cities_df["closest country code"] = ""

for index, row in cities_df.iterrows():
    city = citipy.nearest_city(row["latitude"], row["longitude"])
    cities_df.set_value(index, "closest city name", city.city_name)
    cities_df.set_value(index, "closest country code", city.country_code)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [ ]:
clean_cities_df = cities_df.drop(['latitude', 'longitude'], axis = 1)
clean_cities_df = clean_cities_df.drop_duplicates()
clean_cities_df.head()

In [ ]:
selected_cities = clean_cities_df.sample(500)
selected_cities = selected_cities.reset_index(drop=True)

In [ ]:
# openweathermap api calls
base_url = "http://api.openweathermap.org/data/2.5/weather?"

app_id = api_key

params = {"appid" : app_id, "units" : "metric"}

query_url = base_url + "appid=" + app_id
print(query_url)


In [ ]:
# api call
for index, row in cities_df.iterrows():
    params["q"] = f'{row["closest city name"]}, {row["closest country code"]}'
    print(f'getting weather info for {params["q"]}')
    city_weather_response = requests.get(base_url, params)
    city_weather_response = city_weather_response.json()
    selected_cities_df.set_value(index, "latitude", city_weather_response.get("coord", {}).get("latitude"))
    selected_cities_df.set_value(index, "longitude", city_weather_response.get("coord", {}).get("longitude"))
    selected_cities_df.set_value(index, "temperature", city_weather_response.get("main", {}).get("temp_max"))
    selected_cities_df.set_value(index, "wind speed", city_weather_response.get("wind" , {}).get("speed"))
    selected_cities_df.set_value(index, "humpidity", city_weather_response.get("humidity", {}).get("humidity"))
    selected_cities_df.set_value(index, "cloudiness", city_weather_response.get("clouds", {}).get("all"))
    time.sleep(2)
    
    

In [ ]:
# scatter plot info
def set_plot_prop(x_title, x_lim, y_title):
    plt.title(f'{y_title} vs {x_title}')
    plt.xlabel(x_title)
    plt.ylabel(y_title)
    plt.grid(True)
    plt.xlim(x_lim)
    

In [ ]:
# temp vs latitude
selected_cities_df.plot(kind="scatter", x="latitude", y="temperature", gride=True, color="blue")
set_plot_prop("latitude", [-90, 90], "temperature(F)")
plt.axvline(0, color='black', alpha=0.5)
plt.savefig("temperature vs latitude")
plt.show()


In [ ]:
# humidity vs latitude
selected_cities_df.plot(kind="scatter", x="latitude", y="humidity", grid=True, color="blue")
set_plot_prop("latitude", [-90, 90], "humidity")
plt.axvline(0, color='black', alpha=0.5)
plt.savefig("humidity vs latitude")
plt.show()

In [ ]:
#  wind speed vs latitude
selected_cities_df["wind speed"] = pd.to_numeric(cities["wind speed"])
selected_cities_df.plot(kind="scatter", x="latitude", y="wind speed", grid=True, color="blue")
set_plot_prop("latitude", [-90, 90], "wind speed (mph)")
plt.axvline(0, color='black', alpha=0.5)
plt.savefig("wind speed vs latitude")
plt.show()


In [ ]:
# cloudiness vs latitude
selected_cities_df["cloudiness"] = pd.to_numeric(cities["cloudiness"])
selected_cities_df.plot(kind="scatter", x="latitude", y="cloudiness", grid=True, color="blue")
set_plot_prop("latitude", [-90,90], "cloudiness")
plt.axvline(0, color='black', alpha=0.5)
plt.savefig("cloudiness vs latitude")
plt.show()


In [ ]:
# temperature getting closer to equator
size_temp = np.round((((cities.temperature)/100**2)*10,2))
selected_cities.plot(kind="scatter", x="longitude", y="latitude", grid=True, edgecolor="black", c=size_temp, s=200, cmap=plt.cmY10rRd)
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.tight_layout()
plt.title("temperature getting closer to equator")
plt.grid(True)
plt.xlim([-200,200])
plt.subplots_adjust(bottom=.25, left=.25)
plt.ylim([-85,85])
plt.axhline(0, color='black', alpha=0.5)
plt.savefig("temperature getting closer to equator")
plt.show()
